# Using the Train Matchbox Model by Azure ML Studio

In [2]:
import urllib
import json
import requests as rq
import os
import pandas as pd
import numpy as np
import warnings
import json
from google.cloud import bigquery
from google.oauth2 import service_account

warnings.filterwarnings('ignore')

In [3]:
credentials = service_account.Credentials.from_service_account_file(os.getcwd()+'/final-project-team2-3547e307824d.json')
project_id = 'final-project-team2'
client = bigquery.Client(credentials= credentials,project=project_id)

query = ("SELECT * FROM movielens.customers")
query_job = client.query(
    query,# Location must match that of the dataset(s) referenced in the query.
    location="US",)  # API request - starts the query
customer = query_job.to_dataframe()
customer.head()


query = ("SELECT * FROM movielens.movies")
query_job = client.query(
    query,# Location must match that of the dataset(s) referenced in the query.
    location="US",)  # API request - starts the query
movies = query_job.to_dataframe()
movies.head()

url = 'https://ussouthcentral.services.azureml.net/workspaces/b5735f278ad44ea1b4f08ede6d289e5f/services/a1bf6e157b234b3bb5512a034b671763/execute?api-version=2.0&details=true'
api_key = 'API_KEY' # Replace this with the API key for the web service


firstnames = []
lastnames = []
emails = []
mydictionary = dict()

for index, row in customer.iterrows():
    userId = row['user_id']
    firstnames.append({userId:row['first_name']})
    lastnames.append({userId:row['last_name']})
    emails.append({userId:row['email_id']})
        
tmp = customer.set_index('user_id').T.to_dict('list')

for key in sorted(tmp.keys()):
    mydictionary.update({key:tmp[key]})
    
moviesList = []

for index,row in movies.iterrows():
    movieId = row['movie_id']
    moviesList.append({movieId:row['movie_title']})

def recommendMovie(userId):
    data = {
        "Inputs":{
            "input1":
                {
                    "user_id": ["userId"],
                    "Values" : [[userId]]
                },
        },
        "GlobalParameters":{

        }
    }

    body = str.encode(json.dumps(data))
    
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
    
    req = urllib.request.Request(url, body, headers)
#     print(req)
    response = urllib.request.urlopen(req)
    result = response.read().decode('utf-8')
#     print (result)
    d = json.loads(result)
#     print(d)
#     print (d['Results']['output1']['value']['Values'])
    dlist = d['Results']['output1']['value']['Values']
    finalList = []
    for i in dlist:
        for j in range(len(i)):
            if j == 0:
                continue
            else:
                finalList.append(int(i[j]))
    return finalList
  
    

In [4]:
recommendMovie(10)

[318, 296, 2571, 2959, 527]

In [5]:
recommendMovie(520)

[1217, 318, 750, 1203, 1104]

In [6]:
recommendMovie(612)

[1217, 318, 750, 720, 1104]